# Host Your Pretrained Models on SageMaker in 3 Steps
There are a lot of advantages to developing and training your models on SageMaker. (1) It's fast and easy, so you can iterate quickly to experiment with new packages and ideas, then deploy these to your customers. (2) You can version all of your experiments, then replicate them months down the road after you've forgotten the specifics. (3) You can decouple your development from your training environment, and scale these separately. This lets you optimize your compute and, more importantly, your time.

But let's say you've got a deadline, and you don't want to re-think your training environment right now. You just want to deploy what you have, and with as little overhead as possible. This is the example for you! 

## 1. Put your pretrained models into `container/scripts/models`
We're going to create a container around your pretrained models to simplify the experience of bringing a model trained outside of SageMaker. For this notebook, just make sure you have them placed in the `container/scripts/models` directory.

Everything inside of `container/scripts` is getting copied into SageMaker. So if you have a file in the same directory as `container/scripts` on your notebook instance, it's going to show up in the same directory on the SageMaker endpoint cluster. 

In [1]:
!ls container/scripts/models

model.pkl  transformer.pkl


## 2. Write your prediction code

#### Add your requirements to the container
First, let's get your framework and package needs defined. The Dockerfile we're using has quite a few packages already installed in your image; they're right here.
```python3 
nginx 
ca-certificates 
numpy==1.16.2 scipy==1.2.1 pandas flask gevent gunicorn```
In the cell below, feel free to add any additional package requirements right here. This example will add them to the base Docker image for you. 

You can fully configure how this code works, so as long as you can install the software in your Docker container, you can run it on SageMaker. It does not matter which framework you are using, or what version that is. You just need to set that when you build the image with `container/Dockerfile`.

This example is using sklearn, but you can modify this to point to whichever framework you need.

In [2]:
%%writefile requirements.txt
scikit_learn==0.20.3

Overwriting requirements.txt


Next, import the function `add_to_base_container` from the `utils.py` package available with this example. That will create a new Docker file around your package needs.

In [3]:
from utils import add_to_base_container
                
add_to_base_container('requirements.txt')

Parsing your file requirements.txt
Created your target file container/Dockerfile


#### Add your model to the container
Here you'll define how to pick up your model artifacts from disk.

This example is assuming that you're providing both a `model` and a `transformer` object, which can take your new data and transform it to the correct specifications for your model. We'll pick up both the model and the transformer, and add them to the container. Here they get added to a `base_predictor.py` file to create the final `container/scripts/predictor.py` file that will run on the endpoint.

Remember - SageMaker is going to run this script __from inside the `container/script` directory__. That means you want to point to model files referencing them from there. Your notebook is above that in the directory, so if you want to test your models in this notebook before writing the script, make sure to change the path in the `pickle.load` method.

In [4]:
%%writefile my_script.py
import pickle

model = pickle.load(open('models/model.pkl', 'rb'))
transformer = pickle.load(open('models/transformer.pkl', 'rb'))


Overwriting my_script.py


## 3. Deploy it
Here, just pass in the name of your script. If you're doing this iteratively to test new versions, you can pass in a new version name to track its progress.

If you want to change `utils.py` you are welcome to! Just remember, every time you make a change there you need to restart the kernel for this notebook so it incorporates the latest changes. 

Make sure you're chaing `image_version` as your developing your solution. The SageMaker API likes to see unique model names, so we need to actually create a new model name everytime we want to point to a new image.

In [1]:
from utils import deploy
import os

image_version = '6'

model_base_name = 'shareable'

deploy('my_script.py', image_version, model_base_name)

Got the details from your script, my_script.py
Adding to target file: container/scripts/predictor.py
cd container && docker tag scripts 023375022819.dkr.ecr.us-east-2.amazonaws.com/scripts:shareable-6
cd container && docker push 023375022819.dkr.ecr.us-east-2.amazonaws.com/scripts:shareable-6
Deploying your endpoint: shareable-6-endpoint


## 4. Test it
Now, let's test your endpoint to make sure it's handling data the way you expect it to. You'll need to wait 7+ minutes to test it, because SageMaker is actually creating a live RESTful API around your model when we say `.deploy()`. Alternatively, you can point to an endpoint that already exists.

In [2]:
endpoint_name = '{}-{}-endpoint'.format(model_base_name, image_version)
print (endpoint_name)

shareable-4-endpoint


In [3]:
import csv
import numpy as np

pred_array = np.array([[ 1,  2],
       [ 1,  4],
       [ 1,  0],
       [10,  2],
       [10,  4],
       [10,  0]])

with open('test_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(pred_array)

In [22]:
import boto3

from sagemaker.predictor import csv_serializer

client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=csv_serializer(open('test_data.csv')))

y_pred = response['Body'].read().decode('utf-8').split('\n')

In [ ]:
y_pred